In [1]:
from incremental_td import incremental_TD
import numpy as np
import time

In [20]:
nrep = 1

# gen data from mech1
m, T, r = 200, 100, 10
sigma = 0.1
np.random.seed(5)
Ut, R = np.linalg.qr(np.random.normal(size = (m, r)))
Vt, R = np.linalg.qr(np.random.normal(size = (m, r)))
M_mean = Ut.dot(np.diag([r-i for i in range(r)])).dot(Vt.T)
Tensor = np.zeros((m, m, T))
for i in range(T):
	Tensor[:,:,i] = M_mean + np.random.normal(scale = sigma, size = (m, m))

# prepare starting points for DTA and TD
# reset r here
r = 10
Ut = Ut[:,:r]
Vt = Vt[:,:r]
Us, d, Vs = np.linalg.svd(Tensor[:,:,1])
Us = Us[:,:r]
Vs = Vs[:,:r]

In [21]:
#####################################################################
###   ITD                                                         ###
###   ITD                                                         ###
###   ITD                                                         ###
#####################################################################
nrep = 50

time_record = np.zeros(nrep)
mean_acc = np.zeros(nrep)

for rep in range(nrep):
	# holder for accuracy by T
	acc = np.zeros(T)

	# regenerate data
	for i in range(T):
		Tensor[:,:,i] = M_mean + np.random.normal(scale = sigma, size = (m, m))
	
	# set starting point
	U, V = Us, Vs
	s1 = s2 = np.ones(r)

	start_time = time.time()
	for i in range(T):
		s1, U, s2, V = incremental_TD(s1, U, s2, V, Tensor[:,:,i], i+1, tol=1e-7)
		M_hat = np.linalg.multi_dot([U, U.T, Tensor[:,:,i], V, V.T])
		acc[i] = np.linalg.norm(Tensor[:,:,i] - M_hat) ** 2 / np.linalg.norm(Tensor[:,:,i]) ** 2
	time_record[rep] = time.time() - start_time #;print(acc)
	mean_acc[rep] = acc.mean()


In [22]:
print(mean_acc.mean())
print(mean_acc.std())

print(time_record.mean())
print(time_record.std())

0.51022714813
0.00125749258811
0.493349714279
0.0671085808792


In [23]:
def dtavec(U, s, x, f = 1):
	n = x.shape[1]
	r = U.shape[1]
	for i in range(n):
		xx = x[:, i]
		for j in range(r):
			y = np.dot(U[:, j], xx)
			s[j] = f * s[j] + y ** 2
			e = xx - y * U[:, j]
			U[:, j] += y / s[j] * e
			xx -= y * U[:, j]
			U[:, j] = U[:, j] / np.linalg.norm(U[:, j])
		U_new, R = np.linalg.qr(U)
		if not np.allclose(U_new[:,0], U[:, 0]):
			U_new[:,0] = -U_new[:,0]
	return U_new, s

def dta(s, U, s2, V, x, f = 1):
	U_new, s = dtavec(U, s, x, f)
	V_new, s2 = dtavec(V, s2, x.T, f)
	return s, U_new, s2, V_new

In [24]:
nrep = 10
#####################################################################
###   DTA                                                         ###
###   DTA                                                         ###
###   DTA                                                         ###
#####################################################################

time_record = np.zeros(nrep)
mean_acc = np.zeros(nrep)

for rep in range(nrep):
	# holder for accuracy by T
	acc = np.zeros(T)

	# regenerate data
	for i in range(T):
		Tensor[:,:,i] = M_mean + np.random.normal(scale = sigma, size = (m, m))
	
	# set starting point
	U, V = Us, Vs
	s1 = s2 = 0.1 * np.ones(r)

	start_time = time.time()
	for i in range(T):
		s1, U, s2, V = dta(s1, U, s2, V, Tensor[:,:,i], f = 1)
		M_hat = np.linalg.multi_dot([U, U.T, Tensor[:,:,i], V, V.T])
		acc[i] = np.linalg.norm(Tensor[:,:,i] - M_hat) ** 2 / np.linalg.norm(Tensor[:,:,i]) ** 2
	time_record[rep] = time.time() - start_time
	mean_acc[rep] = acc.mean()

In [25]:
print(mean_acc.mean())
print(mean_acc.std())

print(time_record.mean())
print(time_record.std())


0.999993340627
2.67558519213e-06
24.8578066111
1.01197323855


In [26]:
#####################################################################
###   GTRUTH                                                      ###
###   GTRUTH                                                      ###
###   GTRUTH                                                      ###
#####################################################################
nrep = 20
time_record = np.zeros(nrep)
mean_acc = np.zeros(nrep)

for rep in range(nrep):
	# holder for accuracy by T
	acc = np.zeros(T)

	# regenerate data
	for i in range(T):
		Tensor[:,:,i] = M_mean + np.random.normal(scale = sigma, size = (m, m))

	U, V = Ut, Vt
	start_time = time.time()
	for i in range(T):
		M_hat = np.linalg.multi_dot([U, U.T, Tensor[:,:,i], V, V.T])
		acc[i] = np.linalg.norm(Tensor[:,:,i] - M_hat) ** 2 / np.linalg.norm(Tensor[:,:,i]) ** 2
	time_record[rep] = time.time() - start_time
	mean_acc[rep] = acc.mean()

In [27]:
print(mean_acc.mean())
print(mean_acc.std())

0.508225116946
0.000283430264528


In [28]:
print(U.T @ Ut)

[[  1.00000000e+00   1.25767452e-17  -3.29597460e-17   6.93889390e-18
   -2.25514052e-17   1.99493200e-17  -2.16840434e-18  -2.77555756e-17
    1.75640752e-17  -1.38777878e-17]
 [  1.25767452e-17   1.00000000e+00   1.21430643e-17   1.73472348e-18
    2.16840434e-17   2.36356074e-17  -2.42861287e-17  -2.77555756e-17
    0.00000000e+00  -3.46944695e-17]
 [ -3.29597460e-17   1.21430643e-17   1.00000000e+00   0.00000000e+00
    3.46944695e-18   2.25514052e-17   4.11996826e-18   2.58040117e-17
    1.04083409e-17   1.04083409e-17]
 [  6.93889390e-18   1.73472348e-18   0.00000000e+00   1.00000000e+00
   -8.67361738e-18   8.67361738e-18   5.05238212e-17  -1.38777878e-17
   -1.04083409e-17  -8.32667268e-17]
 [ -2.25514052e-17   2.16840434e-17   3.46944695e-18  -8.67361738e-18
    1.00000000e+00  -6.93889390e-17  -4.07660017e-17  -1.21430643e-17
   -9.97465999e-18   0.00000000e+00]
 [  1.99493200e-17   2.36356074e-17   2.25514052e-17   8.67361738e-18
   -6.93889390e-17   1.00000000e+00  -3.46944